<a href="https://colab.research.google.com/github/raulbenitez/DEEPCARDIO/blob/master/sportsClassifier/sports_classifier_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the necessary packages
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", required=True,
	help="path to trained serialized model")
ap.add_argument("-l", "--label-bin", required=True,
	help="path to  label binarizer")
ap.add_argument("-i", "--input", required=True,
	help="path to our input video")
ap.add_argument("-o", "--output", required=True,
	help="path to our output video")
ap.add_argument("-s", "--size", type=int, default=128,
	help="size of queue for averaging")
args = vars(ap.parse_args())

In [10]:
video_name = 'field_hockey'
args = {"dataset": 'drive/My Drive/tfm/learn/SportsClassifier/Sports-Type-Classifier/data/',
        "model": 'drive/My Drive/tfm/learn/SportsClassifier/model/activity.model',
        "label_bin": 'drive/My Drive/tfm/learn/SportsClassifier/model/lb.pickle',
        "input": f'drive/My Drive/tfm/learn/SportsClassifier/video/{video_name}.mov',
        "output": f'drive/My Drive/tfm/learn/SportsClassifier/video/{video_name}_o.mov',
        "size": 128,
        "epochs": 25}

## Load model

In [4]:
# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(args["model"])
lb = pickle.loads(open(args["label_bin"], "rb").read())

[INFO] loading model and label binarizer...


## Label videos

In [5]:
def label_video(args):
  # initialize the image mean for mean subtraction along with the
  # predictions queue
  mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
  Q = deque(maxlen=args["size"])

  # initialize the video stream, pointer to output video file, and
  # frame dimensions
  vs = cv2.VideoCapture(args["input"])
  writer = None
  (W, H) = (None, None)
  # loop over frames from the video file stream
  while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
      break
    # if the frame dimensions are empty, grab them
    if W is None or H is None:
      (H, W) = frame.shape[:2]
    
    # clone the output frame, then convert it from BGR to RGB
    # ordering, resize the frame to a fixed 224x224, and then
    # perform mean subtraction
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean

    # make predictions on the frame and then update the predictions
    # queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)
    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]

    # draw the activity on the output frame
    text = "activity: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
      1.25, (0, 255, 0), 5)
    # check if the video writer is None
    if writer is None:
      # initialize our video writer
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      writer = cv2.VideoWriter(args["output"], fourcc, 30,
        (W, H), True)
    # write the output frame to disk
    writer.write(output)
    # show the output image
    # cv2.imshow("Output", output)
    # key = cv2.waitKey(1) & 0xFF
    # if the 'q' key was pressed, break from the loop
    # if key == ord("q"):
    # 	break
  # release the file pointers
  print("[INFO] cleaning up...")
  writer.release()
  vs.release()

In [11]:
label_video(args)

[INFO] cleaning up...


In [12]:
auxargs = args.copy()
auxargs['size'] = 1
auxargs['output'] = f'drive/My Drive/tfm/learn/SportsClassifier/video/{video_name}_o2.mov'
label_video(auxargs)

[INFO] cleaning up...


In [9]:
!ls "drive/My Drive/tfm/learn/SportsClassifier/video"

field_hockey.mov   ice_hockey_o.mov	  weight_lifting_o.mov
ice_hockey.mov	   weight_lifting.mov
ice_hockey_o2.mov  weight_lifting_o2.mov
